In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
import numpy as np

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## **Load SAS database files**

In [2]:
with io.capture_output() as captured: main_df = pd.read_sas('~/Desktop/Rush/HCRIS/hosp10-sas/prds_hosp10_yr2010.sas7bdat')
main_df['FFY'] = ['2010'] * main_df.shape[0]
main_df = main_df.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
main_df['rpt_rec_num'] = main_df['rpt_rec_num'].astype(int)
main_df['rpt_rec_num'] = main_df['rpt_rec_num'].astype(str)
main_df = main_df.replace(r"^ +| +$", r"", regex=True)
main_df.columns = main_df.columns.str.strip()
print('2010: (rows, columns) =', main_df.shape)
print(len(main_df['prvdr_num'].unique()), main_df.shape[0], '\n')

yrs = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
for yr in yrs:
    with io.capture_output() as captured: tdf = pd.read_sas('~/Desktop/Rush/HCRIS/hosp10-sas/prds_hosp10_yr' + yr + '.sas7bdat')
    tdf['FFY'] = [yr] * tdf.shape[0]
    tdf = tdf.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
    tdf['rpt_rec_num'] = tdf['rpt_rec_num'].astype(int)
    tdf['rpt_rec_num'] = tdf['rpt_rec_num'].astype(str)
    tdf = tdf.replace(r"^ +| +$", r"", regex=True)
    tdf.columns = tdf.columns.str.strip()
    main_df = pd.concat([main_df, tdf], ignore_index=True)
    print(yr + ': (rows, columns) =', main_df.shape)
    print(len(tdf['prvdr_num'].unique()), len(tdf['rpt_rec_num'].unique()), tdf.shape[0], '\n')
    
del tdf

2010: (rows, columns) = (2323, 5528)
2323 2323 

2011: (rows, columns) = (8473, 5528)
6097 6150 6150 

2012: (rows, columns) = (14700, 5528)
6132 6227 6227 

2013: (rows, columns) = (20948, 5528)
6114 6248 6248 

2014: (rows, columns) = (27198, 5528)
6123 6250 6250 

2015: (rows, columns) = (33455, 5528)
6124 6257 6257 

2016: (rows, columns) = (39666, 5528)
6107 6211 6211 

2017: (rows, columns) = (45840, 5528)
6076 6174 6174 

2018: (rows, columns) = (52000, 5528)
6048 6160 6160 

2019: (rows, columns) = (58121, 5528)
6013 6121 6121 

2020: (rows, columns) = (64179, 5528)
5988 6058 6058 

2021: (rows, columns) = (70229, 5528)
5977 6050 6050 

2022: (rows, columns) = (73084, 5528)
2842 2855 2855 

2023: (rows, columns) = (73087, 5528)
3 3 3 



In [3]:
print(main_df.shape[1], 'features')
main_df.dropna(how='all', axis=1, inplace=True)
print(main_df.shape[1], 'features after removing those with no data')

5528 features
3909 features after removing those with no data


In [4]:
to_replace = ['rpt_rec_num', 'prvdr_num', 'fi_num', 'rpt_stus_cd', 'fi_creat_dt', 
              'fy_bgn_dt', 'fy_end_dt', 'util_cd', 'trnsmtl_num', 'state', 
              'st_cty_cd', 'census', 'region', 'proc_dt', 'msa', 'H3_1_HHA1_C10_15____',
              'H3_1_HHA1_C11_15____', 'sub3', 
             ]

replacement = ['RPT_REC_NUM', 'PRVDR_NUM', 'FI_NUM', 'RPT_STUS_CD', 'FI_CREAT_DT', 
               'FY_BGN_DT', 'FY_END_DT', 'UTIL_CODE', 'TRNSMTL_NUM', 'STATE', 
               'ST_CTY_CD', 'CENSUS', 'REGION', 'PROC_DT', 'MSA', 'H3_1_HHA1_C10_15',
               'H3_1_HHA1_C11_15', 'SUB3',
              ]

main_df.rename(columns = {to_replace[0]: replacement[0], to_replace[1]: replacement[1],
                           to_replace[2]: replacement[2], to_replace[3]: replacement[3],
                           to_replace[4]: replacement[4], to_replace[5]: replacement[5],
                           to_replace[6]: replacement[6], to_replace[7]: replacement[7],
                           to_replace[8]: replacement[8], to_replace[9]: replacement[9],
                           to_replace[10]: replacement[10], to_replace[11]: replacement[11],
                           to_replace[12]: replacement[12], to_replace[13]: replacement[13],
                           to_replace[14]: replacement[14], to_replace[15]: replacement[15],
                           to_replace[16]: replacement[16], to_replace[17]: replacement[17],
                         }, inplace = True)

print('main_df.shape:', main_df.shape)

main_df.shape: (73087, 3909)


In [5]:
main_df['RPT_REC_NUM'] = main_df['RPT_REC_NUM'].astype(int).astype(str)
main_df.sort_values(by=['PRVDR_NUM', 'FFY'], inplace=True, ascending=True)
print(main_df.shape)
main_df.head()

(73087, 3909)


RPT_REC_NUM PRVDR_NUM FI_NUM RPT_STUS_CD FI_CREAT_DT  FY_BGN_DT  \
7988       642305    010001  10001           3  2019-02-06 2010-10-01   
14068      642306    010001  10001           2  2019-02-05 2011-10-01   
19978      642307    010001  10001           2  2019-02-05 2012-10-01   
25535      642308    010001  10001           2  2019-02-05 2013-10-01   
33219      724514    010001  10001           4  2022-09-15 2014-10-01   

       FY_END_DT UTIL_CODE TRNSMTL_NUM STATE ST_CTY_CD  CENSUS  REGION  \
7988  2011-09-30         F           A    01     01340     6.0     4.0   
14068 2012-09-30         F           B    01     01340     6.0     4.0   
19978 2013-09-30         F           C    01     01340     6.0     4.0   
25535 2014-09-30         F           E    01     01340     6.0     4.0   
33219 2015-09-30         F           G    01     01340     6.0     4.0   

         PROC_DT   MSA  A_C1_114  A_C2_114  A_C2_61  B_1_C21_100  B_1_C21_94  \
7988  2019-02-07  2180       NaN       NaN      NaN          NaN         NaN   
14068 2019-02-07  2180       NaN       NaN      NaN          NaN         NaN   
19978 2019-02-07  2180       NaN       NaN      NaN          NaN         NaN   
25535 2019-02-07  2180       NaN       NaN      NaN          NaN         NaN   
33219 2022-09-19  2180       NaN       NaN      NaN          NaN         NaN   

       B_1_C22_100  B_1_C22_42  B_1_C22_94  B_1_C26_200  C_1_C6_100  \
7988           NaN         NaN         NaN          NaN         NaN   
14068          NaN         NaN         NaN          NaN         NaN   
19978          NaN         NaN         NaN          NaN         NaN   
25535          NaN         NaN         NaN          NaN         NaN   
33219          NaN         NaN         NaN          NaN         NaN   

       C_1_C7_100  C_1_C7_111  D3_HOS_C1_77  D3_HOS_C2_201  D3_HOS_C2_42  \
7988          NaN         NaN           NaN            NaN           NaN   
14068         NaN         NaN           NaN            NaN           NaN   
19978         NaN         NaN           NaN            NaN           NaN   
25535         NaN         NaN           NaN            NaN           NaN   
33219         NaN         NaN           NaN            NaN           NaN   

       D3_HOS_C2_43  D3_HOS_C2_61  D3_HOS_C2_77  D3_HOS_C3_61  D3_HOS_C3_77  \
7988            NaN           NaN           NaN           NaN           NaN   
14068           NaN           NaN           NaN           NaN           NaN   
19978           NaN           NaN           NaN           NaN           NaN   
25535           NaN           NaN           NaN           NaN           NaN   
33219           NaN           NaN           NaN           NaN           NaN   

       D3_IPF_C2_42  D3_IPF_C2_94  D3_IPF_C3_94  D3_IRF_C2_61  D3_IRF_C2_94  \
7988            NaN           NaN           NaN           NaN           NaN   
14068           NaN           NaN           NaN           NaN           NaN   
19978           NaN           NaN           NaN           NaN           NaN   
25535           NaN           NaN           NaN           NaN           NaN   
33219           NaN           NaN           NaN           NaN           NaN   

       D3_IRF_C3_61  D3_IRF_C3_94  D3_SNF_C2_40  D3_SNF_C2_41  \
7988            NaN           NaN           NaN           NaN   
14068           NaN           NaN           NaN           NaN   
19978           NaN           NaN           NaN           NaN   
25535           NaN           NaN           NaN           NaN   
33219           NaN           NaN           NaN           NaN   

       D_1_HOS_C4_4201  D_2_IPF_C5_94  D_2_IRF_C5_94  D_4_HOS_C12_94  \
7988               NaN            NaN            NaN             NaN   
14068              NaN            NaN            NaN             NaN   
19978              NaN            NaN            NaN             NaN   
25535              NaN            NaN            NaN             NaN   
33219              NaN            NaN            NaN    

In [6]:
crosswalk_df = pd.read_csv('~/GitHub/HCRIS-databuilder/crosswalk/2552-10 SAS FILE RECORD LAYOUT AND CROSSWALK TO 96 - 2021.csv', sep=',')
crosswalk_features = crosswalk_df['10_FIELD_NAME'].tolist()
crosswalk_features = [str(x).strip(' ') for x in crosswalk_features]
crosswalk_df['10_FIELD_NAME'] = crosswalk_features

crosswalk_df = crosswalk_df.replace(r"^ +| +$", r"", regex=True)
crosswalk_df.columns = crosswalk_df.columns.str.strip()

print(crosswalk_df.shape[0], 'features in the crosswalk')
print('\nNumber of features in the crosswalk:', len(crosswalk_features))
print('Number of unique features in the crosswalk:', len(list(set(crosswalk_features))))

repeated = set([x for x in crosswalk_features if crosswalk_features.count(x) > 1])

print(repeated)

5569 features in the crosswalk

Number of features in the crosswalk: 5569
Number of unique features in the crosswalk: 5518
{'NOT ON FORM', 'S2_1_C1_116', 'H4_2_HHA1_C1_3101', 'S2_1_C1_22', 'nan'}


In [7]:
tdfc = crosswalk_df.copy(deep=True)
print(tdfc.shape)
tdfc = tdfc[~tdfc['FIELD DESCRIPTION'].isin(['', np.nan, float('NaN')])]
print(tdfc.shape[0], 'labeled features, i.e., features with descriptive names')


(5569, 10)
3802 labeled features, i.e., features with descriptive names


In [8]:
main_df_col_labels = list(main_df)
main_df_col_labels = [str(x).strip(' ') for x in main_df_col_labels]

print('\nNumber of features in the crosswalk:', len(crosswalk_features))
print('Number of unique features in the crosswalk:', len(list(set(crosswalk_features))))

print('\nNumber of features in the main_df:', len(main_df_col_labels))
print('Number of unique features in the main_df:', len(list(set(main_df_col_labels))))

shared_labels = list(set(main_df_col_labels) & set(crosswalk_features))
print('\nNumber of features shared between the main dataframe and the crosswalk:', len(shared_labels))
shared_labels.append('File Date')

dif = set(main_df_col_labels).difference(crosswalk_features)
print('\n' + str(len(dif)) + ' labels in main dataframe but not in crosswalk:')
print(dif)

dif = set(crosswalk_features).difference(main_df_col_labels)
print('\n' + str(len(dif)) + ' labels in crosswalk but not in main dataframe:')

del main_df_col_labels
del dif


Number of features in the crosswalk: 5569
Number of unique features in the crosswalk: 5518

Number of features in the main_df: 3909
Number of unique features in the main_df: 3909

Number of features shared between the main dataframe and the crosswalk: 3894

15 labels in main dataframe but not in crosswalk:
{'_NAME_', 'E_A_HOS_C101_2901', 'S3_1_C8_3201', 'E_A_HOS_C1_203', 'S2_1_C2_22', 'S3_1_C14_4', 'S3_1_C14_3', 'PROC_DT', 'E_A_HOS_C1_204', 'S3_1_C14_2', 'E_A_HOS_C101_203', 'H4_2_HHA3_C1_3101', 'D1_HOS_C1_47', 'E_A_HOS_C101_204', 'FFY'}

1624 labels in crosswalk but not in main dataframe:


In [9]:
main_df = main_df[main_df.columns[main_df.columns.isin(shared_labels + ['FFY'])]]
del shared_labels

main_df = main_df.dropna(axis=1, how='all')
main_df = main_df.dropna(axis=0, how='all')
print('main_df.shape (columns and rows with no data removed):', main_df.shape)

main_df.shape (columns and rows with no data removed): (73087, 3895)


In [10]:

CODE = []
FIELD_DESCRIPTION = []
TYPE = []
COMBINED_CODE_FIELD_DESCRIPTION = []

col_labels = list(main_df)

for lab in col_labels:
    if lab == 'FFY':
        CODE.append('FFY')
        FIELD_DESCRIPTION.append('FFY')
        TYPE.append('FFY')
        COMBINED_CODE_FIELD_DESCRIPTION.append('FFY')
        
    else:
        df_sub = crosswalk_df[crosswalk_df['10_FIELD_NAME'] == lab]
        CODE.append(df_sub['10_FIELD_NAME'].iloc[0])
        
        x1 = df_sub['FIELD DESCRIPTION'].iloc[0]
        if x1 == "" or pd.isnull(x1):
            x1 = ''
        FIELD_DESCRIPTION.append(x1)
        
        x2 = df_sub['TYPE'].iloc[0]
        if x2 == "" or pd.isnull(x2):
            x2 = ''
        TYPE.append(x2)
        
        x3 = df_sub['SUBTYPE'].iloc[0]
        if x3 == "" or pd.isnull(x3):
            x3 = ''
        
        if x1 == '' and x3 == '':
            COMBINED_CODE_FIELD_DESCRIPTION.append(lab)
        elif x1 == '':
            COMBINED_CODE_FIELD_DESCRIPTION.append(x3 + ' (' + lab + ')')
        elif x3 == '':
            COMBINED_CODE_FIELD_DESCRIPTION.append(x1 + ' (' + lab + ')')
        else:
            COMBINED_CODE_FIELD_DESCRIPTION.append(x3 + ', ' + x1 + ' (' + lab + ')')

df2 = pd.DataFrame([col_labels, FIELD_DESCRIPTION, TYPE, COMBINED_CODE_FIELD_DESCRIPTION], columns=col_labels)


In [11]:
df2.head(10)

RPT_REC_NUM                             PRVDR_NUM  \
0                          RPT_REC_NUM                             PRVDR_NUM   
1                 Report Record Number              Hospital Provider Number   
2  HOSPITAL IDENTIFICATION INFORMATION   HOSPITAL IDENTIFICATION INFORMATION   
3   Report Record Number (RPT_REC_NUM)  Hospital Provider Number (PRVDR_NUM)   

                                FI_NUM                          RPT_STUS_CD  \
0                               FI_NUM                          RPT_STUS_CD   
1           Fiscal Intermediary Number                   Report Status Code   
2  HOSPITAL IDENTIFICATION INFORMATION  HOSPITAL IDENTIFICATION INFORMATION   
3  Fiscal Intermediary Number (FI_NUM)     Report Status Code (RPT_STUS_CD)   

                                         FI_CREAT_DT  \
0                                        FI_CREAT_DT   
1                Fiscal Intermediary/MAC Create Date   
2                HOSPITAL IDENTIFICATION INFORMATION   
3  Fiscal Intermediary/MAC Create Date (FI_CREAT_DT)   

                             FY_BGN_DT                            FY_END_DT  \
0                            FY_BGN_DT                            FY_END_DT   
1               Fiscal Year Begin Date                 Fiscal Year End Date   
2  HOSPITAL IDENTIFICATION INFORMATION  HOSPITAL IDENTIFICATION INFORMATION   
3   Fiscal Year Begin Date (FY_BGN_DT)     Fiscal Year End Date (FY_END_DT)   

                              UTIL_CODE                          TRNSMTL_NUM  \
0                             UTIL_CODE                          TRNSMTL_NUM   
1              Utilization Code (L/N/F)                     NOT IN THIS FILE   
2   HOSPITAL IDENTIFICATION INFORMATION  HOSPITAL IDENTIFICATION INFORMATION   
3  Utilization Code (L/N/F) (UTIL_CODE)       NOT IN THIS FILE (TRNSMTL_NUM)   

                                 STATE                            ST_CTY_CD  \
0                                STATE                            ST_CTY_CD   
1                       SSA State Code                SSA State County Code   
2  HOSPITAL IDENTIFICATION INFORMATION  HOSPITAL IDENTIFICATION INFORMATION   
3               SSA State Code (STATE)    SSA State County Code (ST_CTY_CD)   

                                CENSUS                               REGION  \
0                               CENSUS                               REGION   
1                 Census Division Code                      CMS Region Code   
2  HOSPITAL IDENTIFICATION INFORMATION  HOSPITAL IDENTIFICATION INFORMATION   
3        Census Division Code (CENSUS)             CMS Region Code (REGION)   

                                   MSA                           A_C1_114  \
0                                  MSA                           A_C1_114   
1                       Geographic MSA             Utilization Review SNF   
2  HOSPITAL IDENTIFICATION INFORMATION  TRIAL BALANCE OF EXPENSE ACCOUNTS   
3                 Geographic MSA (MSA)  Utilization Review SNF (A_C1_114)   

                            A_C2_114  \
0                           A_C2_114   
1             Utilization Review SNF   
2  TRIAL BALANCE OF EXPENSE ACCOUNTS   
3  Utilization Review SNF (A_C2_114)   

                                            A_C2_61  \
0                                           A_C2_61   
1            PBP Clinical Lab Services Program Only   
2                 TRIAL BALANCE OF EXPENSE ACCOUNTS   
3  PBP Clinical Lab Services Program Only (A_C2_61)   

                                         B_1_C21_100  \
0                                        B_1_C21_100   
1           Int & Res (non approved teaching program   
2                                    COST ALLOCATION   
3  Int & Res (non approved teaching program (B_1_...   

                           B_1_C21_94  \
0                          B_1_C21_94   
1               Home Program Dialysis   
2                     COST ALLOCATION   
3  Home Program Dialysis (B_1_C21_94)   

          

In [12]:
main_df = pd.concat([df2, main_df])
print(main_df.shape)

(73091, 3895)


In [13]:
main_df2 = main_df.copy(deep=True)
print(main_df2.shape)

ls = np.array(main_df2.iloc[[1]]).tolist()
ls = ls[0]
print(len(ls))

(73091, 3895)
3895


In [14]:
# Remove features with no label, i.e., no field description in the crosswalk
to_remove = []
labs = list(main_df2)
for i, lab in enumerate(labs):
    if ls[i] == '':
        to_remove.append(lab)
        
print(len(to_remove))

df2 = main_df2.drop(labels=to_remove, axis=1)
print(df2.shape[1], 'features remaining')

1107
2788 features remaining


In [15]:
main_df = df2.copy(deep=True)
del df2
del main_df2

main_df.columns = pd.MultiIndex.from_arrays(main_df.iloc[0:4].values)
main_df = main_df.iloc[4:]

del col_labels
del FIELD_DESCRIPTION
del TYPE
del COMBINED_CODE_FIELD_DESCRIPTION

main_df.head()
print(main_df.shape, 'features remaining')

(73087, 2788) features remaining


In [16]:
CMS_Gen_Info_df = pd.read_csv('~/GitHub/HCRIS-databuilder/GeoData/Hospital_General_Information.tsv', sep='\t')
print(list(CMS_Gen_Info_df))
CMS_Gen_Info_df.head()

['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'ZIP Code', 'County Name', 'Phone Number', 'Hospital Type', 'Hospital Ownership', 'Emergency Services', 'Meets criteria for promoting interoperability of EHRs', 'Hospital overall rating', 'Hospital overall rating footnote', 'Mortality national comparison', 'Mortality national comparison footnote', 'Safety of care national comparison', 'Safety of care national comparison footnote', 'Readmission national comparison', 'Readmission national comparison footnote', 'Patient experience national comparison', 'Patient experience national comparison footnote', 'Effectiveness of care national comparison', 'Effectiveness of care national comparison footnote', 'Timeliness of care national comparison', 'Timeliness of care national comparison footnote', 'Efficient use of medical imaging national comparison', 'Efficient use of medical imaging national comparison footnote', 'Location']


Facility ID                              Facility Name  \
0      030084  CHINLE COMPREHENSIVE HEALTH CARE FACILITY   
1      044022                   CONWAY BEHAVIORAL HEALTH   
2      10021F             96th Medical Group (Eglin AFB)   
3      170204                ROCK REGIONAL HOSPITAL, LLC   
4      171322                   HAMILTON COUNTY HOSPITAL   

                           Address       City State  ZIP Code County Name  \
0        US HWY 191, HOSPITAL ROAD     CHINLE    AZ     86503      APACHE   
1                2255 STURGIS ROAD     CONWAY    AR     72034    FAULKNER   
2  96 MDG 307 Boatner Rd Suite 114  Eglin AFB    FL     32542    OKALOOSA   
3             3251 NORTH ROCK ROAD      DERBY    KS     67037    SEDGWICK   
4                  700 NORTH HUSER   SYRACUSE    KS     67878    HAMILTON   

     Phone Number                       Hospital Type     Hospital Ownership  \
0  (928) 674-7001                Acute Care Hospitals   Government - Federal   
1  (501) 205-0011                         Psychiatric            Proprietary   
2  (850) 883-8600  Acute Care - Department of Defense  Department of Defense   
3  (833) 345-7625                Acute Care Hospitals            Proprietary   
4  (620) 384-7461           Critical Access Hospitals     Government - Local   

   Emergency Services Meets criteria for promoting interoperability of EHRs  \
0                True                                               True      
1               False                                                NaN      
2                True                                                NaN      
3                True                                                NaN      
4                True                                                NaN      

  Hospital overall rating  Hospital overall rating footnote  \
0                       4                              17.0   
1           Not Available                              19.0   
2           Not Available                              22.0   
3           Not Available                               5.0   
4           Not Available                              16.0   

  Mortality national comparison  Mortality national comparison footnote  \
0                 Not Available                                     5.0   
1                 Not Available                                    19.0   
2                 Not Available                                    22.0   
3                 Not Available                                     5.0   
4                 Not Available                                    16.0   

  Safety of care national comparison  \
0                      Not Available   
1                      Not Available   
2                      Not Available   
3                      Not Available   
4                      Not Available   

   Safety of care national comparison footnote  \
0                                          5.0   
1                                         19.0   
2                                         22.0   
3                                          5.0   
4                                         16.0   

  Readmission national comparison  Readmission national comparison footnote  \
0      Above the national average                                       NaN   
1                   Not Available                                      19.0   
2                   Not Available                                      22.0   
3                   Not Available                                       5.0   
4                   Not Available                                      16.0   

  Patient experience national comparison  \
0           Same as the national average   
1                          Not Available   
2                          Not Available   
3                          Not Available   
4                          Not Available   

   Patient experience national comparison footnote  \
0                                              NaN   
1 

In [17]:
hospital_ls = main_df.iloc[:, (main_df.columns.get_level_values(0) == 'S2_1_C1_3')].T.values.tolist()[0]
id_ls = main_df.iloc[:, (main_df.columns.get_level_values(0) == 'PRVDR_NUM')].T.values.tolist()[0]

lats = []
lons = []
Htypes = []
Ctypes = []

misses = 0
hits = 0
for i, h in enumerate(hospital_ls):
    
    fid = id_ls[i]
    
    try:
        df = CMS_Gen_Info_df[CMS_Gen_Info_df['Facility ID'] == fid]
            
        loc = df['Location'].iloc[0]
        loc = loc.replace("POINT (","") 
        loc = loc.replace(")","")
        loc = loc.split(" ")
            
        lat = loc[1]
        lon = loc[0]
        lats.append(lat)
        lons.append(lon)
        
        htype = df['Hospital Type'].iloc[0]
        Htypes.append(htype)
        ctype = df['Hospital Ownership'].iloc[0]
        Ctypes.append(ctype)
        
        del df
        
    except:
        lats.append(float('NaN'))
        lons.append(float('NaN'))
        Htypes.append(float('NaN'))
        Ctypes.append(float('NaN'))


In [18]:
state_codes_df = pd.read_csv('~/GitHub/HCRIS-databuilder/GeoData/HCRIS_STATE_CODES.csv', sep=',')

prvdr_nums = main_df[('PRVDR_NUM', 'Hospital Provider Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)')].tolist()
states1 = main_df[('S2_1_C2_2', 'Hospital State', '', 'Hospital State (S2_1_C2_2)')].tolist()
states2 = []

present = 0
missing = 0
unknown = []

for i, p in enumerate(prvdr_nums):
    
    #print(p)
    code = int(p[0:2])
    
    if code == 66:
        states2.append('MP')
        
    elif states1[i] == states1[i]:
        states2.append(states1[i])
        present += 1
        
    else:
        missing += 1
            
        tdf = state_codes_df[state_codes_df['Ssa_State_Cd'] == code]

        if tdf.shape[0] == 0:
            states2.append('UNKNOWN')
            unknown.append(p)

        else:
            state = tdf['State_Abbreviation'].iloc[0]
            states2.append(state)
        
print('unknown:', unknown)
print('present:', present)
print('missing:', missing)

tdf = main_df.copy(deep=True)
tdf[('S2_1_C2_2', 'Hospital State', '', 'Hospital State (S2_1_C2_2)')] = states2

tdf = tdf.filter(items=[('PRVDR_NUM', 'Hospital Provider Number', 
                         'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)'),
                        ('S2_1_C2_2', 'Hospital State', '', 'Hospital State (S2_1_C2_2)')
                        ])

print('performing check ...')

prvdr_nums = tdf[('PRVDR_NUM', 'Hospital Provider Number', 
             'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)')].tolist()

ct = 0
for p in prvdr_nums:
    tdf2 = tdf[tdf[('PRVDR_NUM', 'Hospital Provider Number', 
             'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)')] == p]
    states = tdf2[('S2_1_C2_2', 'Hospital State', '', 'Hospital State (S2_1_C2_2)')].unique()
    if len(states) > 1:
        ct += 1
        print('hospital no.', p, 'has multiple values for their US state/territory')
        print(states, '\n')
        break

print(ct, 'hospitals with multiple values for their US state/territory')

if ct == 0:
    main_df[('S2_1_C2_2', 'Hospital State', '', 'Hospital State (S2_1_C2_2)')] = states2
    
tdf.head()

unknown: []
present: 73077
missing: 0
performing check ...
0 hospitals with multiple values for their US state/territory


PRVDR_NUM                  S2_1_C2_2
                  Hospital Provider Number             Hospital State
       HOSPITAL IDENTIFICATION INFORMATION                           
      Hospital Provider Number (PRVDR_NUM) Hospital State (S2_1_C2_2)
7988                                010001                         AL
14068                               010001                         AL
19978                               010001                         AL
25535                               010001                         AL
33219                               010001                         AL

In [19]:
for i, val in enumerate(lats):
    if pd.isnull(val): 
        continue
    else:
        print(val, lons[i], Htypes[i], Ctypes[i])
    if i > 10:
        break

31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government - Hospital District or Authority
31.214058 -85.361725 Acute Care Hospitals Government -

In [20]:
print(list(main_df))

[('RPT_REC_NUM', 'Report Record Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Report Record Number (RPT_REC_NUM)'), ('PRVDR_NUM', 'Hospital Provider Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)'), ('FI_NUM', 'Fiscal Intermediary Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Intermediary Number (FI_NUM)'), ('RPT_STUS_CD', 'Report Status Code', 'HOSPITAL IDENTIFICATION INFORMATION', 'Report Status Code (RPT_STUS_CD)'), ('FI_CREAT_DT', 'Fiscal Intermediary/MAC Create Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Intermediary/MAC Create Date (FI_CREAT_DT)'), ('FY_BGN_DT', 'Fiscal Year Begin Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Year Begin Date (FY_BGN_DT)'), ('FY_END_DT', 'Fiscal Year End Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Year End Date (FY_END_DT)'), ('UTIL_CODE', 'Utilization Code (L/N/F)', 'HOSPITAL IDENTIFICATION INFORMATION', 'Utilization Code (L/N/F) (UTIL_CODE)'), ('TRNSMTL_NUM', 'NOT IN

In [21]:
main_df[('Lat', 'Lat', 'Lat', 'Lat')] = lats
main_df[('Lon', 'Lon', 'Lon', 'Lon')] = lons
main_df[('Control type, text', 'Control type, text', 'Control type, text', 'Control type, text')] = Ctypes
main_df[('Hospital type, text', 'Hospital type, text', 'Hospital type, text', 'Hospital type, text')] = Htypes
main_df[('Name and Num', 'Name and Num', 'Name and Num', 'Name and Num')] = main_df[('S2_1_C1_3', 'Hospital Name', '', 'Hospital Name (S2_1_C1_3)')].astype(str) +' (' + main_df[('PRVDR_NUM', 'Hospital Provider Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)')].astype(str) + ')'   
main_df[('Name and Num', 'Name and Num', 'Name and Num', 'Name and Num')] = main_df[('Name and Num', 'Name and Num', 'Name and Num', 'Name and Num')].str.replace("#","No.")

del lats
del lons
del Ctypes
del Htypes


## Correct for inconsistent hospital names

In [22]:
name_and_nums = sorted(main_df[('Name and Num', 'Name and Num', 'Name and Num', 'Name and Num')].unique())

#with open("name_and_number.txt", 'w') as f:
#    f.write("\n".join(map(str, name_and_nums)))

num_name = main_df[('Name and Num', 'Name and Num', 'Name and Num', 'Name and Num')].unique()
print(len(num_name), 'unique hospital name and number combinations')

nums = main_df[('PRVDR_NUM', 'Hospital Provider Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)')].unique()
print(len(nums), 'unique CMS numbers')
    

9865 unique hospital name and number combinations
6908 unique CMS numbers


In [23]:
nn_df = pd.read_csv('~/GitHub/HCRIS-databuilder/name_and_number.csv', sep=',',
                   names=['original label', 'curated label'])

print(len(nn_df['original label'].unique()), 'original name and number combinations')
print(len(nn_df['curated label'].unique()), 'curated name and number combinations')
nn_df.head() 


9754 original name and number combinations
8558 curated name and number combinations


original label  \
0                                 Original Label   
1   1125 SIR FRANCIS DRAKE OPERATING CO (052043)   
2  4499 ACUSHNET AVENUE OPERATING COMPM (222043)   
3  4499 ACUSHNET AVENUE OPERATING COMPA (222043)   
4            A.G. HOLLEY STATE HOSPITAL (102000)   

                                curated label  
0                               Curated Label  
1  KENTFIELD REHABILITATION HOSPITAL (052043)  
2    VIBRA HOSP OF SOUTHEASTERN MASS (222043)  
3    VIBRA HOSP OF SOUTHEASTERN MASS (222043)  
4         A.G. HOLLEY STATE HOSPITAL (102000)

In [24]:
import numpy as np

o_labs = nn_df['original label'].tolist()
c_labs = nn_df['curated label'].tolist()

new_nns = []
ct = 0
for nn1 in main_df[('Name and Num', 'Name and Num', 'Name and Num', 'Name and Num')].tolist():
    
    try:
        i = o_labs.index(nn1)
        nn2 = c_labs[i]
    except:
        print(nn1, 'not in list')
    
    #if nn1 != nn2 and ct < 10: # this is just to demonstrate a few naming errors that get fixed
    #    print(nn1, ' | ', nn2, '\n')
    #    ct += 1
    
    if nn2 in [np.nan, float('NaN')]:
        print('nn2:', nn2)
        break
        
    new_nns.append(nn2)
    
print(main_df.shape[0])
print(len(new_nns))

main_df[('Curated Name and Num', 'Curated Name and Num', 'Curated Name and Num', 'Curated Name and Num')] = new_nns

cols = ['Lat', 'Lon', 'Control type, text', 'Hospital type, text', 'Curated Name and Num']

for c in cols:
    mc = (c, c, c, c)
    # shift column to first position
    col = main_df.pop(mc)
    main_df.insert(0, mc, col)

WHITFIELD REGIONAL HOSPITAL (010112) not in list
MARICOPA COMMUNITY HOSPITAL (030152) not in list
UNITY HEALTH - NEWPORT (040163) not in list
UNITY HEALTH - NEWPORT (041332) not in list
COMMUNITY HOSPITAL LONG BEACH (050849) not in list
COALINGA REGIONAL MEDICAL CENTER (051338) not in list
ALISO RIDGE BEHAVIORAL HEALTH (054158) not in list
SACRAMENTO BEHAVIORAL HEALTHCARE (054159) not in list
CENTURA ST. ELIZABETH HOSPITAL (060044) not in list
FAMILY HEALTH WEST HOSPITAL (061302) not in list
VIBRA HOSPITAL DENVER (062014) not in list
PAM SPEC. HOSPITAL OF WESTMINSTER (063037) not in list
HCA FLORIDA HIGHLANDS HOSPITAL (100049) not in list
HCA FLORIDA PASADENA HOSPITAL (100126) not in list
HCA FLORIDA AVENTURA HOSPITAL (100131) not in list
HCA FLORIDA LAKE CITY HOSPITAL (100156) not in list
HCA FLORIDA MERCY HOSPITAL (100167) not in list
HCA FLORIDA KENDALL HOSPITAL (100209) not in list
HCA FLORIDA OCALA HOSPITAL (100212) not in list
HCA FLORIDA PUTNAM HOSPITAL (100232) not in list
HCA 

In [25]:
tdf = main_df.filter(items=[('Lat', 'Lat', 'Lat', 'Lat'),
                             ('Lon', 'Lon', 'Lon', 'Lon'),
                             ('Control type, text', 'Control type, text', 'Control type, text', 'Control type, text'),
                             ('Hospital type, text', 'Hospital type, text', 'Hospital type, text', 'Hospital type, text'),
                             ('Curated Name and Num', 'Curated Name and Num', 'Curated Name and Num', 'Curated Name and Num'),
                             ('PRVDR_NUM', 'Hospital Provider Number', 'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)'),
                             ('S3_1_C2_27', 'Total Facility', 'NUMBER OF BEDS', 'Total Facility (S3_1_C2_27)'),
                             ('S2_1_C2_2', 'Hospital State', '', 'Hospital State (S2_1_C2_2)'),
                            ], axis=1)

tdf.to_pickle('GenDat4App_p4.pkl', protocol=4)

del tdf

In [26]:
main_df[('FY_END_DT', 'Fiscal Year End Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Year End Date (FY_END_DT)')] = pd.to_datetime(main_df[('FY_END_DT', 'Fiscal Year End Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Year End Date (FY_END_DT)')])
main_df = main_df.sort_values(by=[('Curated Name and Num', 'Curated Name and Num', 'Curated Name and Num', 'Curated Name and Num'),
                     ('FY_END_DT', 'Fiscal Year End Date', 'HOSPITAL IDENTIFICATION INFORMATION', 'Fiscal Year End Date (FY_END_DT)')],
                     ascending=[True, True])


In [27]:
main_df.to_pickle('HCRIS_p4.pkl.gz', protocol=4, compression='infer')

print(main_df.shape[1], 'features')
print(len(main_df[('PRVDR_NUM', 'Hospital Provider Number', 
             'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)')].unique()), 'hospitals')


2794 features
6908 hospitals


In [28]:

report_categories = list(set(main_df.columns.get_level_values(2).tolist()))
report_categories = [x for x in report_categories if str(x) != 'nan']
report_categories = [x for x in report_categories if str(x) != 'Lat']
report_categories = [x for x in report_categories if str(x) != 'Lon']
report_categories = [x for x in report_categories if str(x) != 'Curated Name and Num']
report_categories = [x for x in report_categories if str(x) != 'Hospital type, text']
report_categories = [x for x in report_categories if str(x) != 'Control type, text']
report_categories = [x for x in report_categories if str(x) != 'HOSPITAL IDENTIFICATION INFORMATION']
report_categories.sort()

import csv

with open('report_categories.csv', 'w+', newline='') as OUT:
    writer = csv.writer(OUT)
    writer.writerow(report_categories)
    
    
sub_categories = list(set(main_df.columns.get_level_values(3).tolist()))
sub_categories.sort()

with open('sub_categories.csv', 'w+', newline='') as OUT:
    writer = csv.writer(OUT)
    writer.writerow(sub_categories)


In [29]:
import re

prvdr_nums = sorted(main_df[('PRVDR_NUM', 'Hospital Provider Number', 
             'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)')].unique())

for i, val in enumerate(prvdr_nums):
    
    tdf = main_df[main_df[('PRVDR_NUM', 'Hospital Provider Number', 
             'HOSPITAL IDENTIFICATION INFORMATION', 'Hospital Provider Number (PRVDR_NUM)')] == val]
    
    states1 = tdf[('S2_1_C2_2', 'Hospital State', '', 'Hospital State (S2_1_C2_2)')].unique()
    states2 = [x for x in states1 if x == x]
    
    if len(list(set(states2))) > 1:
        print('Error: hospital CMS number is listed as being in 2 states!')
        print('CMS #:', val)
        print(states1)
        print(states2)
        break
    
    elif len(states2) == 0:
        print('Error: No state given. Fixing ...')
        print('CMS #:', val)
        tdf2 = CMS_Gen_Info_df[CMS_Gen_Info_df['Facility ID'] == val]
        print(tdf2.shape)
        state = tdf2['State']
        print('state:', state)
        tdf[('S2_1_C2_2', 'Hospital State', '', 'Hospital State (S2_1_C2_2)')] = [state] * tdf.shape[0]
        break
        
    elif len(states1) != len(states2):
        print('CMS #:', val)
        print(states1)
        print(states2)
        tdf[('S2_1_C2_2', 'Hospital State', '', 'Hospital State (S2_1_C2_2)')] = [states2[0]] * tdf.shape[0]
        
    tdf.to_csv('provider_data/' + val + '.csv')

del tdf